## RQA analysis examples

The following script contains examples for running RQA on categorical and continuous data. 

Before running any analyses, we need to import and load the relevant packages and utilities:

In [2]:
import os
import pandas as pd
from utils import filter_data, interpolate_missing_data, perform_rqa
from utils.rqa_utils import perform_rqa
from utils.plot_utils import plot_rqa, plot_ts_and_rqa, plot_windowed_ts_and_rqa

Let's start by running RQA on a categorical time series:

In [ ]:
# Set the directory path for the data
data_directory = "data/rqaCategorical/"

# Save image or not
save_image = True

# Set the file name for categorical data
file_name = "DriftingSystem.txt"
file_path = os.path.join(data_directory, file_name)

# RQA parameters
rqa_params = {
    'delay': 1,
    'embedding_dimension': 1,
    'radius': 0.1,
    'minLine': 2,
    'getRP': True
}

# Check if the file exists
if os.path.exists(file_path):
    print(f'Loading file: {file_name}')

    # Load the CSV file into a DataFrame
    data = pd.read_csv(file_path, header=None)

    # Assume the data is a single unnamed column
    category_data = data.iloc[:, 0]

    # Convert categorical data to numerical values (e.g., label encoding)
    categories = category_data.unique()
    category_mapping = {category: idx for idx, category in enumerate(categories)}
    numerical_data = category_data.map(category_mapping).to_frame()

    # Perform RQA using perform_rqa function from rqa_utils
    rqa_results, rp_results = perform_rqa(numerical_data, **rqa_params)

    # Print RQA results
    for column, result in rqa_results.items():
        print(f'RQA results for column {column}:')
        print(f'Recurrence Rate: {result.recurrence_rate}')
        print(f'Determinism: {result.determinism}')
        print(f'Maxline: {result.longest_diagonal_line}')
        print(f'Meanline: {result.average_diagonal_line}')
        print(f'Entropy: {result.entropy_diagonal_lines}')
        print(f'Laminarity: {result.laminarity}')

    # Plot time series and recurrence plot side-by-side
    plot_ts_and_rqa(numerical_data[0].values, rp_results[0].recurrence_matrix, save_image, f'images/rqa/{file_name}_combined_plot.png')
    
    print('RQA analysis and plotting completed successfully!')
else:
    print(f'File {file_name} not found in directory {data_directory}')

Now let's run RQA on a continuous time series:

In [ ]:
# Set the directory path for the data
data_directory = "data/rqaContinuous/"

# Save image or not
save_image = True

# Set the file name for categorical data
file_name = "ecg5000.txt"
file_path = os.path.join(data_directory, file_name)

# RQA parameters
rqa_params = {
    'delay': 15,
    'embedding_dimension': 3,
    'radius': 0.1,
    'minLine': 2,
    'getRP': True
}

# Check if the file exists
if os.path.exists(file_path):
    print(f'Loading file: {file_name}')

    # Load the CSV file into a DataFrame
    data = pd.read_csv(file_path, header=None)

    # Assume the data is a single unnamed column
    continuous_data = data.iloc[:, 0]

    # Convert the series to a DataFrame to maintain consistency with filter_data function
    continuous_data = continuous_data.to_frame()

    # Interpolate missing data
    continuous_data = interpolate_missing_data(continuous_data)

    # Filter the data
    continuous_data = filter_data(continuous_data)
    
    # Z-score normalization
    continuous_data = (continuous_data - continuous_data.mean()) / continuous_data.std()

    # Perform RQA using perform_rqa function from rqa_utils
    rqa_results, rp_results = perform_rqa(continuous_data, **rqa_params)

    # Print RQA results
    for column, result in rqa_results.items():
        print(f'RQA results for column {column}:')
        print(f'Recurrence Rate: {result.recurrence_rate}')
        print(f'Determinism: {result.determinism}')
        print(f'Maxline: {result.longest_diagonal_line}')
        print(f'Meanline: {result.average_diagonal_line}')
        print(f'Entropy: {result.entropy_diagonal_lines}')
        print(f'Laminarity: {result.laminarity}')

    # Plot time series and recurrence plot side-by-side
    plot_ts_and_rqa(continuous_data[0].values, rp_results[0].recurrence_matrix, save_image, f'images/rqa/{file_name}_combined_plot.png')
    
    print('RQA analysis and plotting completed successfully!')
else:
    print(f'File {file_name} not found in directory {data_directory}')

Running RQA on a continuous time series requires the selection of hyperparameters, namely the time delay, embedding dimension, and radius. Let's see how different radii influence the RQA plot and resultant metrics:

In [ ]:
# Set the directory path for the continuous data
data_directory = "data/rqaContinuous/"

# Save image or not
save_image = True

# Set the file name for continuous data
file_name = "b1000.txt"
file_path = os.path.join(data_directory, file_name)

# RQA parameters for continuous data
rqa_params = {
    'delay': 15,
    'embedding_dimension': 3,
    'minLine': 2,
    'getRP': True
}

# Radii values to iterate over
radii = [0.1, 0.2, 0.3]

# Check if the file exists
if os.path.exists(file_path):
    print(f'Loading file: {file_name}')

    # Load the TXT file into a DataFrame
    data = pd.read_csv(file_path, header=None)

    # Assume the data is a single unnamed column
    continuous_data = data.iloc[:, 0]

    # Convert the series to a DataFrame to maintain consistency with filter_data function
    continuous_data = continuous_data.to_frame()

    # Interpolate missing data
    continuous_data = interpolate_missing_data(continuous_data)

    # Filter the data
    continuous_data = filter_data(continuous_data)
    
    # Z-score normalization
    continuous_data = (continuous_data - continuous_data.mean()) / continuous_data.std()

    # Iterate over different radii values
    for radius in radii:
        print(f'Running RQA with radius: {radius}')
        rqa_params['radius'] = radius

        # Perform RQA using perform_rqa function from rqa_utils
        rqa_results, rp_results = perform_rqa(continuous_data, **rqa_params)

        # Print RQA results
        for column, result in rqa_results.items():
            print(f'RQA results for column {column}:')
            print(f'Recurrence Rate: {result.recurrence_rate}')
            print(f'Determinism: {result.determinism}')
            print(f'Maxline: {result.longest_diagonal_line}')
            print(f'Meanline: {result.average_diagonal_line}')
            print(f'Entropy: {result.entropy_diagonal_lines}')
            print(f'Laminarity: {result.laminarity}')

        # Plot recurrence plot
        plot_rqa(rp_results[0].recurrence_matrix, save_image, f'images/rqa/{file_name}_rp_plot_radius_{radius}.png')
    
    print('RQA analysis and plotting completed successfully!')
else:
    print(f'File {file_name} not found in directory {data_directory}')

Sometimes researchers wish to run a windowed RQA analysis. This allows us to return a range of RQA metrics and examine their trajectory over time. Let's run this now:

In [ ]:
# Set the directory path for the continuous data
data_directory = "data/rqaContinuous/"

# Save image or not
save_image = True  # Set to True to save plots as images

# Set the file name for continuous data
file_name = "b1000.txt"
file_path = os.path.join(data_directory, file_name)

# RQA parameters for continuous data
rqa_params = {
    'delay': 15,
    'embedding_dimension': 3,
    'radius': 0.1,
    'minLine': 2,
    'getRP': True
}

# Set the frame rate and window size in seconds
fps = 10
window_size_seconds = 60  # Set window size in seconds
window_size = int(window_size_seconds * fps)  # Calculate window size in frames
overlap = 0.5  # 50% overlap
step_size = int(window_size * (1 - overlap))  # Step size for windows

# Check if the file exists
if os.path.exists(file_path):
    print(f'Loading file: {file_name}')

    # Load the TXT file into a DataFrame
    data = pd.read_csv(file_path, header=None)

    # Assume the data is a single unnamed column
    continuous_data = data.iloc[:, 0]

    # Convert the series to a DataFrame to maintain consistency with filter_data function
    continuous_data = continuous_data.to_frame()

    # Iterate over the data in windows with 50% overlap
    for start in range(0, len(continuous_data) - window_size + 1, step_size):
        print(f'Processing window starting at {start}')

        # Get the window of data
        window = continuous_data[start:start + window_size]

        # Interpolate missing data
        window = interpolate_missing_data(window)

        # Filter the data
        window = filter_data(window)
        
        # Z-score normalization
        window = (window - window.mean()) / window.std()

        # Perform RQA on the window
        rqa_results, rp_results = perform_rqa(window, **rqa_params)

        # Plot time series and recurrence plot side-by-side
        plot_windowed_ts_and_rqa(window.iloc[:, 0].values, rp_results[0].recurrence_matrix, save_image, f'images/rqa/{file_name}_window_{start}_combined_plot.png')

        # Print specific RQA results
        for column, result in rqa_results.items():
            print(f'RQA results for column {column}:')
            print(f'Recurrence Rate: {result.recurrence_rate}')
            print(f'Determinism: {result.determinism}')
            print(f'Maxline: {result.longest_diagonal_line}')
            print(f'Meanline: {result.average_diagonal_line}')
            print(f'Entropy: {result.entropy_diagonal_lines}')
            print(f'Laminarity: {result.laminarity}')
    
    print('Windowed RQA analysis and plotting completed successfully!')
else:
    print(f'File {file_name} not found in directory {data_directory}')